Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [ ]:
%cd /content
!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install transformers==4.19.0
!pip3 install -r requirements.txt

!pip3 install einops==0.5.0
!pip3 install rotary_embedding_torch==0.1.5
!pip3 install unidecode==1.3.5

!python3 setup.py install

In [8]:
!apt -y install youtube-dl ffmpeg
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
youtube-dl is already the newest version (2020.03.24-1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.0/157.0 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
%cd /content/tortoise-tts
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
tts = TextToSpeech()

/content/tortoise-tts


100% (1716988501 of 1716988501) |########| Elapsed Time: 0:00:06 Time:  0:00:06


Done.


100% (60938957 of 60938957) |############| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


100% (975620731 of 975620731) |##########| Elapsed Time: 0:00:08 Time:  0:00:08


Done.


 65% (99540992 of 151223901) |#######    | Elapsed Time: 0:00:00 ETA:   0:00:00

In [ ]:
# This is the text that will be spoken.
text = "Joining two modalities results in a surprising increase in generalization! What would happen if we combined them all?"

# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [ ]:
%cd /content/tortoise-tts/tortoise/voices/gec
!yt-dlp -x https://www.youtube.com/watch\?v\=nQW3uXpngzs --format mp4
!ffmpeg -i 100\ gecs\ -\ hand\ crushed\ by\ a\ mallet\ \(Acapella\ -\ Vocals\ only\)\ \[nQW3uXpngzs\].m4a -c:v copy -c:a libmp3lame -q:a 4 100-gecs-hand-crushed-by-a-mallet-ACAPELLLA.mp3


In [ ]:
%cd /content/tortoise-tts/tortoise/voices/gec
!ffmpeg -ss 00:00:00 -t 00:00:10.00 -i 100-gecs-hand-crushed-by-a-mallet-ACAPELLLA.mp3 1.mp3
!ffmpeg -ss 00:00:10 -t 00:00:10.00 -i 100-gecs-hand-crushed-by-a-mallet-ACAPELLLA.mp3 2.mp3
!ffmpeg -ss 00:00:20 -t 00:00:10.00 -i 100-gecs-hand-crushed-by-a-mallet-ACAPELLLA.mp3 3.mp3

In [ ]:
# Tortoise will attempt to mimic voices you provide. It comes pre-packaged
# with some voices you might recognize.

# Let's list all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
%ls tortoise/voices

IPython.display.Audio('100-gecs-hand-crushed-by-a-mallet-ACAPELLLA.mp3')

In [18]:
%cd /content/tortoise-tts/
# Pick one of the voices from the output above
voice = 'gec'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

/content/tortoise-tts


NameError: ignored

In [ ]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.", 
                          voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

In [ ]:
del tts  # Will break other cells, but necessary to conserve RAM if you want to run this cell.

# Tortoise comes with some scripts that does a lot of the lifting for you. For example,
# read.py will read a text file for you.
!python3 tortoise/read.py --voice=train_atkins --textfile=tortoise/data/riding_hood.txt --preset=ultra_fast --output_path=.

IPython.display.Audio('train_atkins/combined.wav')
# This will take awhile..